In [10]:
import numpy as np
import pandas as pd

In [11]:
# Read the CSV file into a DataFrame
df = pd.read_csv('input_game.csv')

# Display the DataFrame
print(df)

        game_id  p1_id  p2_id p1_action p2_action  turn
0             0    174    125     TRUST     CHEAT     1
1             0    174    125     CHEAT     TRUST     2
2             0    174    125     TRUST     TRUST     3
3             0    174    125     TRUST     TRUST     4
4             0    174    125     TRUST     TRUST     5
...         ...    ...    ...       ...       ...   ...
180401     3014    157    169     CHEAT     TRUST    64
180402     3014    157    169     CHEAT     TRUST    65
180403     3014    157    169     CHEAT     TRUST    66
180404     3014    157    169     CHEAT     TRUST    67
180405     3014    157    169     CHEAT     TRUST    68

[180406 rows x 6 columns]


In [20]:
strategy_dict = {} # key is a tuple containing the gameid, player1 id and player2 id

for i in range(3):
    game_data = df[df['game_id'] == i]
    
    if game_data.empty:
        continue

    key_player1 = (i,game_data.iloc[0]['p1_id'])
    key_player2 = (i,game_data.iloc[0]['p2_id'])

    p1_cheat_count = 0
    p2_cheat_count = 0

    p1_cheat_count_list = []
    p2_cheat_count_list = []

    cheats_in_window_p1 = {} #dictionary of dictionaries
    cheats_in_window_p2 = {}

    cheats_in_window_p1[0] = {}
    cheats_in_window_p2[0] = {}

    if game_data.iloc[0]['p1_action'] == "CHEAT":
            p1_cheat_count+=1
            p1_cheat_count_list.append(p1_cheat_count)
    else:
        p1_cheat_count_list.append(p1_cheat_count)

    if game_data.iloc[0]['p2_action'] == "CHEAT":
        p2_cheat_count+=1
        p2_cheat_count_list.append(p2_cheat_count)
    else:
        p2_cheat_count_list.append(p1_cheat_count)

    cheats_in_window_p1[1] = {1:p1_cheat_count}
    cheats_in_window_p2[1] = {1:p2_cheat_count}

    if game_data.iloc[1]['p1_action'] == "CHEAT":
            p1_cheat_count+=1
            p1_cheat_count_list.append(p1_cheat_count)
    else:
        p1_cheat_count_list.append(p1_cheat_count)

    if game_data.iloc[1]['p2_action'] == "CHEAT":
        p2_cheat_count+=1
        p2_cheat_count_list.append(p2_cheat_count)
    else:
        p2_cheat_count_list.append(p1_cheat_count)


    for index, row in game_data.iterrows():

        if(index == 0 or index == 1):
            continue

        cheats_in_window_p1[index] = {(index - j): (p1_cheat_count - p1_cheat_count_list[j-1]) for j in range(1,index)}
        cheats_in_window_p1[index][index] = p1_cheat_count

        cheats_in_window_p2[index] = {(index - j): (p2_cheat_count - p2_cheat_count_list[j-1]) for j in range(1,index)}
        cheats_in_window_p2[index][index] = p2_cheat_count

        #updating cheat counts
        if row['p1_action'] == "CHEAT":
            p1_cheat_count+=1
            p1_cheat_count_list.append(p1_cheat_count)
        else:
            p1_cheat_count_list.append(p1_cheat_count)

        if row['p2_action'] == "CHEAT":
            p2_cheat_count+=1
            p2_cheat_count_list.append(p2_cheat_count)
        else:
            p2_cheat_count_list.append(p1_cheat_count)

    
    #stores in format own and other
    strategy_dict[key_player1] = [cheats_in_window_p1,cheats_in_window_p2]
    strategy_dict[key_player2] = [cheats_in_window_p2,cheats_in_window_p1]

print(strategy_dict)


IndexError: list index out of range

In [23]:
df['p1_action'] = df['p1_action'].replace({'TRUST': 0, 'CHEAT': 1})
df['p2_action'] = df['p2_action'].replace({'TRUST': 0, 'CHEAT': 1})

df

,game_id,p1_id,p2_id,p1_action,p2_action,turn
0,0,174,125,0,1,1
1,0,174,125,1,0,2
2,0,174,125,0,0,3
3,0,174,125,0,0,4
4,0,174,125,0,0,5
...,...,...,...,...,...,...
180401,3014,157,169,1,0,64
180402,3014,157,169,1,0,65
180403,3014,157,169,1,0,66
180404,3014,157,169,1,0,67


In [33]:

temp = df[df['p1_id'] == 125]



for i in range(3014):
    game_data = temp[temp['game_id'] == i]

    if(game_data.empty):
        continue
    else:
        gds.append(game_data)


In [50]:
correlation_matrix = np.full((201,201),-1)

player_game_data_dict = {}

for player_id in range(2):

    gds = []

    temp1 = df[df['p1_id'] == player_id]
    temp2 = df[df['p2_id'] == player_id]

    for game_id in range(3014):

        game_data1 = temp1[temp1['game_id'] == game_id]
        game_data2 = temp2[temp2['game_id'] == game_id]

        if(game_data1.empty and game_data2.empty):
            continue
        elif game_data1.empty:
            gds.append(game_data2)
        elif game_data2.empty:
            gds.append(game_data1)
        else:
            gds.append(game_data1)
            gds.append(game_data2)

    player_game_data_dict[player_id] = gds

for player_id in range(2):

    gds = player_game_data_dict[player_id]

    for gd in gds:
        correlation = np.corrcoef(gd["p1_action"],(gd["p2_action"]))[0][1]
        p1_id = gd["p1_id"].iloc[0] - 1
        p2_id = gd["p2_id"].iloc[0] - 1
        if(np.isnan(correlation)):
            correlation = 0

        correlation_matrix[p1_id, p2_id] = correlation


0
